In [1]:
import squidpy as sq
import os,sys
import pandas as pd
import scanpy as sc
# sc.settings.verbosity = 0
import anndata
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pylab as plt
%matplotlib inline
# %matplotlib notebook
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sc.settings.set_figure_params(dpi=80)

In [2]:
# parameters
indir=os.path.expanduser("~/scratch/VPT/20230810_cellpose1")

In [ ]:
# adata=sq.read.vizgen(path="output/region_0",
#                     counts_file="cell_by_gene.csv",
#                     meta_file="cell_metadata.csv",
#                     transformation_file="micron_to_mosaic_pixel_transform.csv",
#                     library_id="spatial")
# adata

In [3]:
library_id="20230828_spatial"

adata = anndata.read_text(os.path.join(indir,"CellByGeneMatrix.csv"),delimiter=",", first_column_names=True)
cell_meta = pd.read_csv(os.path.join(indir,"cell_metadata.csv"),index_col=0)

blank_genes = np.array(["Blank" in v for v in adata.var_names])
adata.obsm["blank_genes"] = pd.DataFrame(
    adata[:, blank_genes].X.copy(), columns=adata.var_names[blank_genes], index=adata.obs_names
)
adata = adata[:, ~blank_genes].copy()

adata.X = csr_matrix(adata.X)
cell_meta.set_index(cell_meta.index.astype("str"), inplace=True)

adata.obs = pd.merge(adata.obs, cell_meta, how="left", left_index=True, right_index=True)
adata.obsm['spatial'] = adata.obs[["center_x", "center_y"]].values
adata.obs.drop(columns=["center_x", "center_y"], inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jzh/scratch/VPT/20230810_cellpose1/CellByGeneMatrix.csv'

In [ ]:
#transformation_file
# matrix = pd.read_csv("output/region_0/images/micron_to_mosaic_pixel_transform.csv", 
#                      sep=" ", header=None)
# # https://github.com/scverse/squidpy/issues/727
# matrix.columns = matrix.columns.astype(str)
# adata.uns['spatial']={}
# adata.uns['spatial'][library_id]={}
# adata.uns['spatial'][library_id]["scalefactors"] = {"transformation_matrix": matrix}
adata

In [ ]:
sc.pl.embedding(adata, basis="spatial", projection="2d")